In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

mes_da_folha = 7

df = pd.read_csv('.\\ATS - 7-2023\\ats gerados - 2023.csv', names=range(12), delimiter=';')
df_adicionais_mentorh = pd.DataFrame()
df_adicionais_mentorh['Nome'] = [nome[:-8] for indice, nome in enumerate(df[1]) if indice%6 == 0]
df_adicionais_mentorh['Matricula'] = [matricula[-5:] for indice, matricula in enumerate(df[1]) if indice%6 == 0]
adicionais_anteriores = [re.findall('\d+', adicional[-4:-2]) for indice, adicional in enumerate(df[1]) if (indice-2) >= 0 and (indice-2)%6==0]
df_adicionais_mentorh['ATS anterior'] = [int(adicional[0]) if len(adicional) > 0 else 0 for adicional in adicionais_anteriores]
df_adicionais_mentorh['ATS novo'] = [int(adicional[:-2]) for indice, adicional in enumerate(df[8]) if (indice-4) >= 0 and (indice-4)%6==0]
df_adicionais_mentorh['Diferenca ATS'] = df_adicionais_mentorh['ATS novo'] - df_adicionais_mentorh['ATS anterior']


df_SQL = pd.read_csv('.\\ATS - 7-2023\\ats gerados - 2023 SQL.csv', delimiter=';')
matriculas_SQL_fora_do_mentorh = df_SQL[df_SQL['Matricula'].apply(lambda x: str(x) not in list(df_adicionais_mentorh['Matricula']))]
if len(matriculas_SQL_fora_do_mentorh) > 0:
        with open('matriculas para incluir no ATS.txt', 'w') as arquivo:
            for index, row in matriculas_SQL_fora_do_mentorh.iterrows():
                linha = row
                arquivo.write(f"{linha}\n")
        
df_SQL['Novo_Inicio'] = df_SQL['Novo_Inicio'].apply(lambda x: int(str(x)[-7:-5]) if len(str(x))>1 else mes_da_folha)
matriculas_com_afastamento = df_SQL[df_SQL['Novo_Inicio'] != mes_da_folha]
if len(matriculas_com_afastamento) > 0:
        with open('matriculas para retirar o ATS.txt', 'w') as arquivo:
            arquivo.write("Retirar as seguintes matrículas e criar a tarefa no JIRA para lançar no mês correto: \n\n")
            for index, row in matriculas_com_afastamento.iterrows():
                linha = row
                arquivo.write(f"{linha}\n")


In [4]:
from datetime import datetime
from mrh.web.scripts.fopag_ats import fopag_ats
julho = datetime(day=31,month=7,year=2023)
# fopag_ats(usuario='', senha='', data_limite=julho)

False

In [6]:
from datetime import datetime
from mrh.web.scripts.fopag_progressao import fopag_progressao
julho = datetime(day=31,month=7,year=2023)
fopag_progressao(usuario='', senha='', data_limite=julho)

False

In [1]:
from datetime import datetime
from mrh.web.scripts.fopag_ferias_ats_prog import fopag_ferias
julho_inicio = datetime(day=1,month=7,year=2023)
julho_fim = datetime(day=31,month=7,year=2023)
fopag_ferias(usuario='', senha='', data_inicio=julho_inicio, data_fim=julho_fim)

False